In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [5]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/KCI_FI003153549.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 12


In [6]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")


분할된 청크의수: 39


In [11]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=gemini_api_key)


In [13]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)


In [21]:
# 로컬에 벡터스토어 생성 및 저장
FAISS_INDEX_PATH = "faiss_index"
vectorstore.save_local(FAISS_INDEX_PATH)

In [14]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()


In [15]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
# 예시 질의
query = "본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수와 문서 유형별 비율은 어떻게 되나요?"
# query = "Advance RAG 기법이 임상시험 데이터 분석에서 수행하는 주요 역할은 무엇인가요?"
# query = "본 연구에서 Private LLM 성능을 평가하기 위해 사용한 지표 3가지는 무엇인가요?"
# query = "국내에서 LLM을 임상시험에 적용한 대표적인 기관과 그 적용 사례를 2가지 이상 말해보세요."
# query = "ROUGE 평가에서 Private LLM과 ChatGPT의 Recall 값은 각각 얼마였나요?"
retriever.invoke(query)


[Document(id='a497918d-843a-4451-a86c-3c6eb0161917', metadata={'producer': 'ezPDF Builder Supreme', 'creator': '', 'creationdate': '2024-12-27T02:09:00+09:00', 'source': 'data/KCI_FI003153549.pdf', 'file_path': 'data/KCI_FI003153549.pdf', 'total_pages': 12, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-12-27T02:09:00+09:00', 'trapped': '', 'modDate': "D:20241227020900+09'00'", 'creationDate': "D:20241227020900+09'00'", 'page': 1}, page_content='소스 모델을 사용하여 임상 데이터를 분석하고, 이를 통해 \n더 신속하고 정확한 결과를 도출하는 방법을 제안한다. 또\n한 이러한 기술이 의료기기 개발과 임상시험에서 혁신을 \n가져올 수 있는 가능성을 평가하며, 향후 임상시험에서 \nLLM 기술의 확장 가능성도 논의한다.\nLLM 기반 기술은 단순히 데이터를 분석하는 것 이상의 \n의미를 지니고 있다. 임상 데이터에서 발견된 패턴을 바탕\n으로 의사결정을 돕고, 궁극적으로는 임상시험의 전반적인 \n효율성을 높일 수 있는 중요한 도구로 활용될 수 있다.[5] \n이 연구는 LLM 기술이 임상시험뿐만 아니라 다양한 의료 \n분야에서 데이터 분석의 패러다임을 변화시킬 수 있는 가\n능성을 제시한다. 향후 LLM을 활용한 의료기기 개발 및'),
 Document(id='70916812-a55b-4f32-90a3-11937b1cce23', metadata={'producer': 'ezPDF Builder Supreme', 'creator': 

In [16]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)


In [17]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=gemini_api_key)

In [18]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [19]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = query
response = chain.invoke(question)
print(response)


본 연구에서 Private LLM 구축을 위해 수집한 문서는 총 111,954페이지입니다.

문서 유형별 비율은 다음과 같습니다:
*   **규제 문서**: 30%
*   **교육 자료**: 20%
*   **프로토콜 및 보고서**: 25%
*   **의료기기 특화 문서**: 15%
*   **기타**: 10% (윤리위원회 관련 문서, 환자 동의서 템플릿 등)
